# CNN for lesion classification

## 1. Brief CNN theory

A Convolutional Neural Network (CNN, or ConvNet) is a type of **feed-forward** artificial neural network in which the connectivity pattern between its neurons is inspired by the organization of the animal visual cortex.

<img src="images/convnets_cover.png" width="70%" />

> source: https://flickrcode.files.wordpress.com/2014/10/conv-net2.png

### 1.1 Structure of a CNN

> A more detailed overview of what CNNs do would be that you take the image, pass it through a series of convolutional, nonlinear, pooling (downsampling), and fully connected layers, and get an output. As we said earlier, the output can be a single class or a probability of classes that best describes the image. 

source: [1]

#### Convolutional Layer

The first layer in a CNN is always a **Convolutional Layer**.

<img src="images/same_padding_no_strides.gif" width="50%">

#### Typical CNN Structure

A traditional CNN architecture consists of other layers interspaced between convolution layers

<img src="images/Table.png">

#### Pooling layer

After some ReLu layers, **pooling layer** is typically applied.

<img src="images/MaxPool.png" width="80%"/>

Pooling reduces the amount of parameters (helping with computional efficiency) and controls overfitting

## 2. We build one using keras and tensorflow

### 2.1 Preparation

In [1]:
# setup code for this notebook
import numpy as np
import matplotlib.pyplot as plt
from functions import data, Timer
timer = Timer()

# This makes matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Make the notebook reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

#### The ultra sound scan data
- 163 scans total, clinically confirmed as having either bening or malignant (cancerous) lesions 
- 100 scans for training, 63 for testing
- Training data was passed through 7 transformations to give us 800 training images total
- 650 images used for training, 150 for validation
- Testing images not transformed
- Both training and testing images were resized to 224X224 this time round (optimal for inception in this case)
- Raw pngs then converted to numpy arrays and saved

In [2]:
# Import keras libraries
from keras.models import Sequential
from keras.applications.inception_v3 import InceptionV3
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from functions import data, Timer

Using TensorFlow backend.


In [3]:
img_rows, img_cols = 224, 224 # 224, 224 works resized down from 360, 528
color_channels = 3

if K.image_data_format() == 'channels_first':
    input_shape = (color_channels, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, color_channels)
    
print('Input shape', input_shape)

Input shape (224, 224, 3)


In [4]:
# data loader and generator helper methods
from keras.preprocessing.image import ImageDataGenerator

In [19]:
# data readers
base = "J:\\final year project\\code and models\\data\\augmented\\"
train_directory = base+'training'
validation_directory = base+'validation'

batch_size = 8

# normalization
train_generator = ImageDataGenerator(rescale=1./255)
validation_generator = ImageDataGenerator(rescale=1./255)

# this is a generator that will read scans found in
# the train directory, and indefinitely generate
# batches of image data
train_generator = train_generator.flow_from_directory(
        train_directory,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='binary')

# A  similar generator, for validation data
validation_generator = validation_generator.flow_from_directory(
        validation_directory,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='binary')

Found 650 images belonging to 2 classes.
Found 150 images belonging to 2 classes.


In [20]:
# Visualize
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

def visualize(model):
    model.summary()
    SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [21]:
def getModelMemoryUsage(batch_size, model):
    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    total_memory = 4.0*batch_size*(shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return str(gbytes) + " GB" , str(gbytes*2)

### 2.2 Retraining InceptionV3 CNN

In [22]:
# We define the custom inception based model
def buildBaseModel():
    input_tensor = Input(shape=input_shape)
    base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)
    
    # we add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    # we add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    
    #we add a logistic layer for our 2 classes
    predictions = Dense(1, activation='softmax')(x)
    
    # this is our model, a hybrid inceptionv3
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # first we train our custom top layer
    # we freeze all convolutional inceptionv3 layers
    for layer in base_model.layers:
        layer.trainable = False
        
    model.compile(optimizer='adagrad',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [23]:
model = buildBaseModel()

In [24]:
# helpers for checkpointing and early stopping
from keras.callbacks import ModelCheckpoint , EarlyStopping

def trainModel(model, epochs=10, text="Re training inception model",
              file_name='best_cnn_inc_model.h5'):
    best_model_file = file_name
    early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=True) 
    best_model = ModelCheckpoint(best_model_file, verbose=True, save_best_only=True)

    timer.start()
    network_history = model.fit_generator(
            train_generator, 
            steps_per_epoch=200,
            epochs=epochs,
            validation_data = validation_generator,
            validation_steps=50,
            verbose=True,
            callbacks=[best_model])
    timer.stop(text)
    return network_history

In [25]:
# visualize(model)
model.summary()
gpu, ram = getModelMemoryUsage(batch_size, model)
print("GPU Memory:" + gpu + "RAM:" + ram)

1.514 GB


In [27]:
history = trainModel(model, 
                     50, 
                     "Re training on our data with frozen inception layers", "lower.h5")

Epoch 1/50
200/200 [==============================] - 70s 349ms/step - loss: 10.7213 - acc: 0.3275 - val_loss: 10.7452 - val_acc: 0.3260

Epoch 00001: val_loss improved from inf to 10.74517, saving model to best_cnn_inc_model.h5
Epoch 2/50
200/200 [==============================] - 69s 344ms/step - loss: 10.6415 - acc: 0.3325 - val_loss: 10.6495 - val_acc: 0.3320

Epoch 00002: val_loss improved from 10.74517 to 10.64951, saving model to best_cnn_inc_model.h5
Epoch 3/50
200/200 [==============================] - 68s 341ms/step - loss: 10.7531 - acc: 0.3255 - val_loss: 10.6495 - val_acc: 0.3320

Epoch 00003: val_loss did not improve
Epoch 4/50
200/200 [==============================] - 68s 342ms/step - loss: 10.5937 - acc: 0.3355 - val_loss: 10.6814 - val_acc: 0.3300

Epoch 00004: val_loss did not improve
Epoch 5/50
200/200 [==============================] - 69s 343ms/step - loss: 10.8010 - acc: 0.3225 - val_loss: 10.5539 - val_acc: 0.3380

Epoch 00005: val_loss improved from 10.64951 to

In [11]:
print(9)

9


In [12]:
# building the rest of the model

# here we choose to retrain the top 2 inception blocks
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# we now recompile the model for our layer modifications to take effect
# we start with SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history_2 = trainModel(model, 
                       50,
                       "Re training the full inception model",
                       '5.h5')

Epoch 1/3
625/625 [==============================] - 650s 1s/step - loss: 10.4830 - acc: 0.3424 - val_loss: 11.3754 - val_acc: 0.2865
Epoch 2/3
625/625 [==============================] - 646s 1s/step - loss: 10.4884 - acc: 0.3421 - val_loss: 11.3941 - val_acc: 0.2853
Epoch 3/3
625/625 [==============================] - 645s 1s/step - loss: 10.4905 - acc: 0.3420 - val_loss: 11.3816 - val_acc: 0.2861
Timing:: took 32 minutes Re training the full inception model


In [13]:
def plot(network_history):
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(network_history.history['loss'])
    plt.plot(network_history.history['val_loss'])
    plt.legend(['Training', 'Validation'])

    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(network_history.history['acc'])
    plt.plot(network_history.history['val_acc'])
    plt.legend(['Training', 'Validation'], loc='lower right')

### 2.3 Evaluating the CNNs performance

In [ ]:
# Get test data
x_test, y_test = data.getTestData()
print('Test data shape: ', x_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
X_test = x_test/255

In [ ]:
# load and evaluate best model
from keras.models import load_model
best_model = load_model('5.h5')
best_model.summary()

In [ ]:
best_model.predict_classes(x_test)

In [ ]:
def printMetrics(tn, fp, fn, tp):
    sensitivity = tp/(tp+fn)
    
    print("True Positive Rate (TPR) or Hit Rate or Recall or Sensitivity: "
          , sensitivity)

In [ ]:
# Evaluate all
from sklearn.metrics import confusion_matrix
expected = y_test
prediction = best_model.predict_classes(x_test)
x, y, fn, tp = confusion_matrix(y_test, prediction).ravel()
printMetrics(fn, tp)

In [ ]:
def printMetrics(tn, fp, fn, tp):
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    error_rate = 1 - accuracy
    precision = tp/(tp+fp)
    f_measure = 2/((1/precision)+(1/sensitivity))
    
    print("True Positive Rate (TPR) or Hit Rate or Recall or Sensitivity: "
          , sensitivity)
    print("False Positive Rate(FPR) or False Alarm Rate: "
          , 1 - specificity)
    print("Accuracy: ", accuracy)
    print("Error rate: ", error_rate)
    print("Precision: ", precision)
    print("F measure: ", f_measure)

In [ ]:
# Evaluate all
from sklearn.metrics import confusion_matrix
expected = y_test
prediction = best_model.predict_classes(x_test)
tn, fp, fn, tp = confusion_matrix(y_test, prediction).ravel()
print(tn, fp, fn, tp)
printMetrics(tn, fp, fn, tp)

### References for images and some content:

\[1\] [https://adeshpande3.github.io/adeshpande3.github.io/]() 
<br> \[2\] ["Neural Networks and Deep Learning"](http://neuralnetworksanddeeplearning.com/) by Michael Nielsen.
<br> \[3\] Deep learning with TensorFlow and Keras by Valerio Maggio